In [1]:
import os, platform
import scipy

distr = platform.dist()[0]

PATH = os.path.expanduser('~/datasets/letsdance') if distr == 'Ubuntu' else '/run/media/nast/DATA/letsdance' 
print("dataset path:", PATH)



/home/anton/datasets/letsdance
